In [1]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [10]:
from datetime import datetime, timedelta
import pandas as pd
import hopsworks  # Import Hopsworks for manual feature store connection

HOPSWORKS_API_KEY = "11Pnc4YExMWVAIq0.11oLgfRGa7dwpPvQkiu6SA0zwpSAhSQjJ0mSgLdg2wKemRB7roS6culWyM51O3x6"
HOPSWORKS_PROJECT_NAME = "nyc_taxi_proj"

# Manually specify feature view details
FEATURE_VIEW_NAME = "time_series_hourly_feature_view"  # Adjust if needed
FEATURE_VIEW_VERSION = 1  # Change based on your feature store setup

# Connect to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
feature_store = project.get_feature_store()
current_date = pd.Timestamp.now(tz="Etc/UTC")

# Read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=29)  # 29 days window
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")

# Get the feature view manually (no config)
feature_view = feature_store.get_feature_view(
    name=FEATURE_VIEW_NAME, version=FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"], inplace=True)
ts_data.reset_index(drop=True, inplace=True)

# Convert timestamp to New York timezone
import pytz
ny_tz = pytz.timezone("America/New_York")
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_convert(ny_tz)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)


2025-03-04 19:09:44,372 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:09:44,378 INFO: Initializing external client
2025-03-04 19:09:44,379 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:09:44,871 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Fetching data from 2025-02-04 00:09:45.272293+00:00 to 2025-03-04 23:09:45.272293+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.43s) 


In [14]:
# from src.inference import get_feature_store
# from datetime import datetime, timedelta
# import pandas as pd  

# # Get the current datetime64[us, Etc/UTC]  
# current_date = pd.Timestamp.now(tz='Etc/UTC')
# feature_store = get_feature_store()

# # read time-series data from the feature store
# fetch_data_to = current_date - timedelta(hours=1)
# fetch_data_from = current_date - timedelta(days=1*29)
# print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
# feature_view = feature_store.get_feature_view(
#     name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
# )

# ts_data = feature_view.get_batch_data(
#     start_time=(fetch_data_from - timedelta(days=1)),
#     end_time=(fetch_data_to + timedelta(days=1)),
# )
# ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
# ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
# ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

# from src.data_utils import transform_ts_data_info_features
# features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [15]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 19:10:41,270 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:10:41,272 INFO: Initializing external client
2025-03-04 19:10:41,272 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:10:41,826 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665


In [16]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [17]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,2,0.0,2025-03-05 01:00:00+00:00
1,3,0.0,2025-03-05 01:00:00+00:00
2,4,2.0,2025-03-05 01:00:00+00:00
3,6,0.0,2025-03-05 01:00:00+00:00
4,7,1.0,2025-03-05 01:00:00+00:00
...,...,...,...
246,259,0.0,2025-03-05 01:00:00+00:00
247,260,0.0,2025-03-05 01:00:00+00:00
248,261,1.0,2025-03-05 01:00:00+00:00
249,262,1.0,2025-03-05 01:00:00+00:00


In [18]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-04 19:10:57,224 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:10:57,226 INFO: Initializing external client
2025-03-04 19:10:57,226 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:10:57,805 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665


In [19]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 251/251 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214665/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)